<a href="https://colab.research.google.com/github/MissHead/SMA-Simulattion/blob/master/SMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema Multiagente para Agenda Inteligente

[P.A.G.E](https://drive.google.com/file/d/1ix2RAKp_IfMEQK0AHOZyVM-2wOaeCvzv/view?usp=sharing)


In [0]:
from random import randint
from time import sleep

In [0]:
class Compromisso:
  compromissos = dict()
  horarios = []
  prioridades = []
  eventos = ["Reunião","Encontro familiar","Salão de Beleza","Consulta Médica",
              "Palestra","Prova","Academia","Igreja","Cinema","Teatro",
              "Workshop","Festa","Aniversário","Encontro","Curso de Inglês"]
  def __init__(self):
    self.compromissos['07:00'] = (0,"Trabalho")
    self.compromissos['17:00'] = (0,"Casa")
    self.compromissos['19:00'] = (0,"Faculdade")
  
    for i in range(7,23):
      if i == 7 or i == 17 or i ==19:
        pass
      else:
        self.horarios.append(str(i).zfill(2) + ":00")
    for i in range(1,9):
      self.prioridades.append(i)
      
    for i in range(0, 25):
      horario = self.horarios[randint(0, len(self.horarios)-1)]
      prioridade = self.prioridades[randint(0, len(self.prioridades)-1)]
      evento = self.eventos[randint(0, len(self.eventos)-1)]
      
      self.compromissos[horario] = (prioridade, evento)

In [0]:
class Agente:
    # A classe Agente possui um diconário global que possui os compromissos de 
    #todos os tipos diferentes de agentes
    agentDict = {}
    
    # O construtor da classe recebe o tipo do agente e inicializa uma chave do 
    #dicionário para listar os compromissos
    def __init__(self, tipo):
      self.tipo = tipo
      self.agentDict[self.tipo] = list()
      
    # Geração dos compromissos de um detemrinado agente de forma randômica    
    def geraCompromissos(self):
      index = randint(0,9)
      for i in range(index):
        for key, value in Compromisso().compromissos.items():
          if value[0] == randint(index,9):
            if (key, value) in self.agentDict[self.tipo]:
              pass
            else:
              self.agentDict[self.tipo].append((key, value))
      if len(self.agentDict[self.tipo]) == 0:
        return CentralMensagens().recebeMensagem(self.tipo, 
                                                 'AgendaPrincipal',
                                                 ('Nenhum Compromisso', 0))
      return CentralMensagens().recebeMensagem(self.tipo, 'AgendaPrincipal', 
                                               ('Novo Compromisso', 
                                                self.agentDict[self.tipo]))

    def geraCompromisso(self, minHora):
      horario = str(randint(minHora, 22)).zfill(2)+':00'
      prioridade = Compromisso().prioridades[randint(0, len(Compromisso().prioridades)-1)]
      evento = Compromisso().eventos[randint(0, len(Compromisso().eventos)-1)]

      mensagem = ('Novo Compromisso', [(horario, (prioridade, evento))])
      print('-> Agente('+self.tipo+'): Mensagem enviada, Destinatário: AgendaPrincipal, Conteúdo: '+str(mensagem))
      CentralMensagens().recebeMensagem(self.tipo, 'AgendaPrincipal', mensagem)

      return mensagem

    

In [0]:
class AgendaPrincipal:
  dia = dict()
  
  # Método que faz a geração automática de um dia somente com os compromissos 
  # de prioridade 0
  def geraDia(self):
    for i in range(7, 23):
      self.dia[str(i).zfill(2)+":00"] = 0
    for key, value in Compromisso().compromissos.items():
      if value[0] == 0:
        self.dia[key] = value
   
  # Método para manipulação do dicionário agenda que controla o dia    
  def insereCompromisso(self, hora, compromisso):
    print('-> AgendaPrincipal: Mensagem Recebida, Novo Compromisso: '+hora+' '+compromisso[1]+' '+'Prioridade '+str(compromisso[0]))
    
    if self.dia[hora] == 0:
      print('-> AgendaPrincipal: Novo compromisso das '+hora+'! Antigo: Nenhum compromisso, Novo: '+compromisso[1])
      self.dia[hora] = compromisso

    elif self.dia[hora][0] == compromisso[0]:
      print('Compromissos com mesma prioridade, favor escolher um:')
      print('1. '+hora+' - '+self.dia[hora][1])
      print('2. '+hora+' - '+compromisso[1])

      ctrl = str(input('Digite o número do compromisso desejado: '))

      if ctrl == '1':
        print('-> AgendaPrincipal: Compromisso original priorizado '+hora+'! Mantido: '+self.dia[hora][1]+'Descartado: '+compromisso[1])
      elif ctrl == '2':
        print('-> AgendaPrincipal: Novo compromisso das '+hora+'! Antigo: '+self.dia[hora][1]+'Novo: '+compromisso[1])
        self.dia[hora] = compromisso
      else:
        print('numero inválido')

    elif self.dia[hora][0] < compromisso[0]:
      print('-> AgendaPrincipal: Novo compromisso das '+hora+'! Antigo: '+self.dia[hora][1]+'Novo: '+compromisso[1])
      self.dia[hora] = compromisso
    
    else:
      print('-> AgendaPrincipal: Compromisso original priorizado '+hora+'! Mantido: '+self.dia[hora][1]+'Descartado: '+compromisso[1])
    
    
    
  
  def getdia(self):
    return self.dia
  
  
  def printdia(self):
    print(self.dia)
   
  def printCronograma(self):
    for k,v in self.dia.items():
      if v == 0:
         print(k+': Sem compromisso!')
      else:
        print(str(k)+': '+v[1]+' Prioridade: '+str(v[0]))


In [0]:
# Classe que serve para fazer a entrega das mensagens aos seus devidos 
# destinatários
class CentralMensagens:
  
  # Audit serve como um histórico de todas as mensagens recebidas
  audit = []
  
  # Método que recebe as mensagens dos agentes
  def recebeMensagem(self, remetente, destinatario, mensagem):
    print('-> Central de Mensagem: Mensagem recebida, remetente: '+remetente+', Conteúdo: '+mensagem[0])
    msg = {}
    msg['remetente'] = remetente
    msg['destinatario'] = destinatario
    msg['mensagem'] = mensagem
    self.audit.append(msg)
    self.enviaMensagem(msg)                  
    #print(self.audit)
            
  # Método que envia os compromissos para a Agenda Principal
  def enviaMensagem(self, msg):
    # É feito uma verificação se há novos compromissos ou não e a central de 
    # mensagem age de acordo
    print('-> Central de Mensagem: Mensagem enviada, Destinatário: AgendaPrincipal, Conteúdo: '+msg['mensagem'][0])
    if msg['mensagem'][0] == 'Novo Compromisso':
      #print(msg['remetente'])
      for item in msg['mensagem'][1]:
        #print('Novo Compromisso: '+item[0]+' - '+str(item[1]))
        AgendaPrincipal().insereCompromisso(hora=item[0], compromisso=item[1])
        #print(AgendaPrincipal().dia)
    if msg['mensagem'][0] == 'Nenhum Compromisso':
      pass
    
  

In [6]:
# Declaração do Agente
c = Agente('email')
s = Agente('smartphone')

# A geração do dia agenda principal tem que se a primeira coisa a ser feita
AgendaPrincipal().geraDia()

# Após gerar o dia da agenda é possível fazer a inserção dos compromissos na 
# agenda do dia

print("--- Novos compromissos ---")
print()
c.geraCompromissos()
s.geraCompromissos()
print()
print("--- Compromissos no dia ---")
print()
AgendaPrincipal().printCronograma()

for i in range(7, 22):
  print()
  print('Hora: '+str(i).zfill(2)+":00")
  compromisso = c.geraCompromisso(i)
  sleep(5)




--- Novos compromissos ---

-> Central de Mensagem: Mensagem recebida, remetente: email, Conteúdo: Novo Compromisso
-> Central de Mensagem: Mensagem enviada, Destinatário: AgendaPrincipal, Conteúdo: Novo Compromisso
-> AgendaPrincipal: Mensagem Recebida, Novo Compromisso: 12:00 Festa Prioridade 6
-> AgendaPrincipal: Novo compromisso das 12:00! Antigo: Nenhum compromisso, Novo: Festa
-> AgendaPrincipal: Mensagem Recebida, Novo Compromisso: 21:00 Palestra Prioridade 6
-> AgendaPrincipal: Novo compromisso das 21:00! Antigo: Nenhum compromisso, Novo: Palestra
-> AgendaPrincipal: Mensagem Recebida, Novo Compromisso: 11:00 Cinema Prioridade 6
-> AgendaPrincipal: Novo compromisso das 11:00! Antigo: Nenhum compromisso, Novo: Cinema
-> AgendaPrincipal: Mensagem Recebida, Novo Compromisso: 13:00 Cinema Prioridade 6
-> AgendaPrincipal: Novo compromisso das 13:00! Antigo: Nenhum compromisso, Novo: Cinema
-> AgendaPrincipal: Mensagem Recebida, Novo Compromisso: 18:00 Curso de Inglês Prioridade 7
->